<a href="https://colab.research.google.com/github/RancheritoMcfly/PropePythonLaunchX/blob/main/Te_damos_la_bienvenida_a_Colaboratory.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [10]:
! pip install mpi4py

     |████████████████████████████████| 2.5 MB 6.6 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for mpi4py: filename=mpi4py-3.1.3-cp37-cp37m-linux_x86_64.whl size=2185250 sha256=ed0600385302d52c7d69b5c45036e8141a70fee1fac3a082da6d423862ac09e2
  Stored in directory: /root/.cache/pip/wheels/7a/07/14/6a0c63fa2c6e473c6edc40985b7d89f05c61ff25ee7f0ad9ac
Successfully built mpi4py


In [14]:
%%writefile Ejemplo.py
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

if rank == 0:
  val_n = 10000

else:
  val_n = None;

dato = comm.bcast(val_n, root = 0)
print( rank, size, dato)

Overwriting Ejemplo.py


In [15]:
! mpirun --allow-run-as-root -np 4 python Ejemplo.py

0 4 10000
1 4 10000
2 4 10000
3 4 10000


In [ ]:
%%writefile Ejemplogather.py
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

if rank != 0:
  lista = [val for val in range(size)]

else:
  lista = None;

dato = comm.gather(lista, root = 0)
print( rank, size, dato)

In [19]:
! mpirun --allow-run-as-root -np 4 python Ejemplogather.py

3 4 None
2 4 None
1 4 None
0 4 [None, [0, 1, 2, 3], [0, 1, 2, 3], [0, 1, 2, 3]]


In [28]:
%%writefile calculoPi.py
from mpi4py import MPI

comm = MPI.COMM_WORLD
rank = comm.Get_rank()
size = comm.Get_size()

def cal_pi(li, ls, step):
  aproxpi = 1
  for k in range(li, ls+1, step):
    k2= k*2
    aproxpi *= (k2**2) / ((k2-1)*(k2+1))
  return aproxpi

if __name__ == "__main__":
  if rank == 0:
    kmax = 2_000_000
  else:
    kmax = None

  km = comm.bcast(kmax, root=0)

  val = cal_pi(rank+1, km, size)
  lista = comm.gather(val, root=0)
  
  if rank == 0:
    val_pi = 2
    for val in lista:
      val_pi *= val
    print(val_pi)

Overwriting calculoPi.py


In [29]:
! mpirun --allow-run-as-root -np 4 python calculoPi.py

3.1415922608922244
